Created on Mon Dec 27 15:14:57 2021
@author: Anurag Srivastava

# Wildfire Ash Model

1. The model computes ash delivery from the hillslope using probabilistic approach with multi-year climate assuming fire happens every year on the same day.
2. The model works with a single OFE simulation of WEPP.
3. The model requires WEPP's daily water file.



Input: WEPP water file ('*_.wat.txt')

Output: Ash transport file ('*_ash.csv')


## Intuition

The WEPP model simulates runoff and sediment yield from different burn severities assuming uniform soil and vegetative characteristics. Following a wildfire, the presence of ash on the forest floor alters the runoff generation mechanism influencing the transport of ash along the hillslope. The interacting climate with eroding ash layer continually changes the physical characteristics of ash over time affecting its capacity to hold the water. The amount of runoff generated from the ash layer depends on ash characteristics (depth of ash, hydraulic conductivity,  bulk density, and particle density), rainfall timing and intensity, and their interactions. 

Following the fire, the presence of water repellent soils at the ash-soil layer interface prevents the percolation of water allowing more water to be stored within the ash layer because of a high porosity (low bulk density) that consequently reduces runoff through the ash layer. Although there is a reduction in runoff through the ash layer, ash particles are more highly erodible with intense rainfall or snowmelt events because of the presence of water-repellent soils underneath.   

As time progresses, successive rainfall or snowmelt events break the water repellent layers letting water percolate through the ash-soil interface, which results in the compaction of the ash layer reducing the porosity (increasing bulk density) of the ash layer. Precipitation events during this phase produce relatively higher runoff generated through the ash layer but become less erodible partly due to the lower amount of ash available for transport and consolidated ash layer.



## Assumptions

- Model operates on daily values of surface runoff and infiltration amount from WEPP.
- Infiltration is a function of rain+melt and surface runoff. Infiltrated water into the top soil layer is based on effective hydraulic conductivity of top mineral soil layer instead of effective hydraulic conductivity of ash layer.
- Bulk density of ash layer increases with time, which is a function of cumulative infiltration since the time of the fire day.
- Water holding capacity (porosity) of ash layer decreases over time with reduced ash depth.


## Model components

### Input Parameters

Following are the input parameters used for ash modeling. Parameter values currently are preliminary.

iniBulkDen = 0.18  # Initial bulk density ($\rho_i$), gm/cm3 

finBulkDen = 0.62  # Final bulk density ($\rho_f$), gm/cm3 

bulkDenFac = 0.005  # Bulk density factor ($\rho_{fact}$)

parDen = 1.2  # Ash particle density ($\rho_m$), gm/cm3 

decompFac = 0.00018  # Ash decomposition factor, t/ha/day

iniErod = 2  # Initial erodibility ($k_i$), t/ha

finErod = 0.01  # Final erodibility ($k_f$), t/ha

roughnessLimit = 1  # Threshold ash depth below which ash delivery is restricted ($R$), mm

iniAshDepth = 14  # Initial ash depth, mm

iniAshLoad = 10 * iniAshDepth * iniBulkDen  # Initial ash load, t/ha


### Infiltration

The ash model relies on modeled WEPP runoff from the burn severity soils and infiltrated amount for the computation of runoff amount from ash layer.

Daily infiltration (I) is computed as a difference of rain plus snowmelt ($RM$) and surface runoff ($Q$).  

$$I = RM - Q$$


### Bulk density


Bulk density is modeled using an exponential function to capture the consolidation of ash layer over time. Bulk density increases exponentially from the bulk density at time 0 (fire day) with increasing cumulative infiltration and asymptotically approaches the bulk density above a certain cumulative infiltration depending on the shape of the curve it follows.

Belk density is computed as:

$$\rho = \rho_f + (\rho_i - \rho_f) \; exp (- {\rho_{fact}}  * {I_{cum}})$$

where:

$\rho$ = bulk density of ash layer (gm/cm<sup>3</sup>)

$\rho_i$ = initial bulk density of ash layer on the day of fire (gm/cm<sup>3</sup>)

$\rho_f$ = asymptotic bulk density approached by the curve under a prolonged period of rainfall or infiltration (gm/cm<sup>3</sup>)

$\rho_{fact}$ = bulk density shape factor that defines the shape of the curve of $\rho$ versus $I_{cum}$ and ensure that bulk density approaches $\rho_f$ 

$I_{cum}$ = cumulative infiltration (mm)


### Porosity

Porosity of the ash layer is computed as a function of bulk density ($\rho$) and particle density. Particle density of the ash within the ash layer is assumed temporally uniform.

$$\Phi = 1 - \frac{\rho_b}{\rho_m}$$

where:

$\Phi$ = porosity of ash layer

$\rho_m$ = particle density of ash (gm/cm<sup>3</sup>)


### Ash runoff

The amount of runoff generated from the ash layer is computed by removing the depth of water that the ash layer can withhold from the runoff amount of WEPP as: 

$$Q_{ash, t-1} = Q_{t-1} - \Bigg(\frac{A_{ash, t-1}} {(10 * \rho_{t-1})}\Bigg) * \Phi$$

where:

$Q_{ash}$ = runoff amount from ash layer (mm)

$Q$ = runoff amount from WEPP (mm)

$A_{ash}$ = available ash amount (t/ha)

10 is conversion factor to convert ash amount in t/ha to ash depth in mm


### Ash delivery

Delivery of ash from the hillslope is calculated as runoff from the ash layer times transportable rate of ash. For higher values of normalized bulk density, ash delivery is governed by maximum ash transport rate while for lower values by asymptotic ash transport rate.  

$$DEL_{ash, t-1} = Q_{ash, t-1} * \Bigg[(k_i - k_f) * \Bigg(\frac{\rho_{t-1} - \rho_f} {\rho_i - \rho_f}\Bigg) + k_f \Bigg]$$

where:

$DEL_{ash}$ = ash delivery from hillslope (t/ha)

$k_i$ = maximum ash transport rate (t/ha/mm)

$k_f$ = asymptotic ash transport rate (t/ha/mm)

$\frac{\rho_{t-1} - \rho_f}{\rho_i - \rho_f}$ = normalized bulk density, ranges between 0-1


### Available ash

The available amount of ash on the current day is calculated by subtracting the ash delivered from the available ash amount on the previous day adjusted for the decomposition factor with infiltration. 


$$A_{ash, t} = A_{ash, t-1} * exp(-decompFac *  I_t) - DEL_{ash, t-1}, \; when \; DEP_{ash, t-1} > R$$
$$A_{ash, t} = 0, \; otherwise$$


where:

$decompFac$ = decay rate of available ash amount with infiltration (t/ha/day)

$DEP_{ash}$ = ash depth (mm)

$R$ = threshold value of ash depth below which ash transport is restricted with the assumption that the remaining ash mixes with the underneath mineral soil (mm) 


### Ash depth

Depth of ash layer for each time-step is computed as:

$$DEP_{ash, t} = \frac{A_{ash, t}} {10 * \rho_{t}} $$


## Model limitations

1) Particle density of ash generally increases as the ash particles become smaller over time. In this model, particle size of ash is assumed uniform. 

2) No evaporation from the ash layer is considered that could affect the runoff amount from the ash layer.

3) Runoff from the ash layer is computed by adjusting the WEPP modeleled runoff for burn severity with porosity. Sophisticated rainfall-infiltration-runoff models such as Green-ampt infiltration model should be considered for reliable estimation of runoff from ash layer.

4) No scheme of simulation of needle leaves over ash layer for moderate and low severity fire.

